<a href="https://colab.research.google.com/github/csanchezcgloria/backend-new/blob/master/clustering_nse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')
ruta= '/content/drive/My Drive/Equifax/20250310_PDV_ENRIQUECIDO.csv'

Mounted at /content/drive


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_src = pd.read_csv(ruta, sep=',',encoding='utf-8')
df_src.head()

,gloria_id,latitud,longitud,ID_BLOCK,PRED,POB,POB_A,POB_B,POB_C,POB_D,POB_E,HOG,HOG_A,HOG_B,HOG_C,HOG_D,HOG_E,ING_A,ING_B,ING_C,ING_D,ING_E,ING,GAS_ALI_A,GAS_BEB_A,GAS_VES_A,GAS_ALQ_A,GAS_MUE_A,GAS_SAL_A,GAS_TRA_A,GAS_COM_A,GAS_REC_A,GAS_EDU_A,GAS_RES_A,GAS_BIE_A,GAS_ALI_B,GAS_BEB_B,GAS_VES_B,GAS_ALQ_B,GAS_MUE_B,GAS_SAL_B,GAS_TRA_B,GAS_COM_B,GAS_REC_B,GAS_EDU_B,GAS_RES_B,GAS_BIE_B,GAS_ALI_C,GAS_BEB_C,GAS_VES_C,GAS_ALQ_C,GAS_MUE_C,GAS_SAL_C,GAS_TRA_C,GAS_COM_C,GAS_REC_C,GAS_EDU_C,GAS_RES_C,GAS_BIE_C,GAS_ALI_D,GAS_BEB_D,GAS_VES_D,GAS_ALQ_D,GAS_MUE_D,GAS_SAL_D,GAS_TRA_D,GAS_COM_D,GAS_REC_D,GAS_EDU_D,GAS_RES_D,GAS_BIE_D,GAS_ALI_E,GAS_BEB_E,GAS_VES_E,GAS_ALQ_E,GAS_MUE_E,GAS_SAL_E,GAS_TRA_E,GAS_COM_E,GAS_REC_E,GAS_EDU_E,GAS_RES_E,GAS_BIE_E,GAS_A,GAS_B,GAS_C,GAS_D,GAS_E,GAS_ALI,GAS_BEB,GAS_VES,GAS_ALQ,GAS_MUE,GAS_SAL,GAS_TRA,GAS_COM,GAS_REC,GAS_EDU,GAS_RES,GAS_BIE,GAS,H_0_4,H_5_9,H_10_14,H_15_19,H_20_24,H_25_29,H_30_34,H_35_39,H_40_44,H_45_49,H_50_54,H_55_59,H_60_64,H_65_69,H_70_74,H_75_79,H_80_MAS,M_0_4,M_5_9,M_10_14,M_15_19,M_20_24,M_25_29,M_30_34,M_35_39,M_40_44,M_45_49,M_50_54,M_55_59,M_60_64,M_65_69,M_70_74,M_75_79,M_80_MAS,GEN_0_4,GEN_5_9,GEN_10_14,GEN_15_19,GEN_20_24,GEN_25_29,GEN_30_34,GEN_35_39,GEN_40_44,GEN_45_49,GEN_50_54,GEN_55_59,GEN_60_64,GEN_65_69,GEN_70_74,GEN_75_79,GEN_80_MAS,H,M,AUSTERO,FORMALISTA,MODERNAS,PROGRESIST,SOFISTICAD,CONSERVADO,PRE_EV
0,G-1,-12.062727,-76.969070,86539,C,85,0,7,44,34,0,25,0,2,13,10,0,0,15230,53768,26330,0,95328,0,0,0,0,0,0,0,0,0,0,0,0,4054,32,358,2110,484,1164,438,1064,292,1120,296,428,20111,65,1456,7956,1872,4472,676,3523,871,2821,1105,1898,11960,20,690,3940,990,1910,340,1320,320,790,460,1020,0,0,0,0,0,0,0,0,0,0,0,0,0,11840,46826,23760,0,36125,117,2504,14006,3346,7546,1454,5907,1483,4731,1861,3346,82426,7,3,2,6,6,8,5,2,1,2,1,0,0,1,0,2,0,3,2,0,0,11,11,3,1,1,3,1,2,1,0,0,0,0,10,5,2,6,17,19,8,3,2,5,2,2,1,1,0,2,0,46,39,1,54,22,34,15,4,FORMALISTAS
1,G-2,-16.390261,-71.536107,663981,A,46,34,6,6,0,0,16,12,2,2,0,0,171468,15230,8272,0,0,194970,26280,420,3180,16488,11268,10176,7728,9192,5508,12204,3228,4044,4054,32,358,2110,484,1164,438,1064,292,1120,296,428,3094,10,224,1224,288,688,104,542,134,434,170,292,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,109716,11840,7204,0,0,33428,462,3762,19822,12040,12028,8270,10798,5934,13758,3694,4764,128760,0,0,0,0,8,10,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,12,11,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,21,0,0,0,0,1,4,0,0,0,0,0,22,24,0,0,0,0,0,0,NaN
2,G-3,-16.393608,-71.519055,663402,C,230,4,51,124,51,0,63,1,14,34,14,0,14289,106610,140624,36862,0,298385,2190,35,265,1374,939,848,644,766,459,1017,269,337,28378,224,2506,14770,3388,8148,3066,7448,2044,7840,2072,2996,52598,170,3808,20808,4896,11696,1768,9214,2278,7378,2890,4964,16744,28,966,5516,1386,2674,476,1848,448,1106,644,1428,0,0,0,0,0,0,0,0,0,0,0,0,9143,82880,122468,33264,0,99910,457,7545,42468,10609,23366,5954,19276,5229,17341,5875,9725,247755,5,6,7,9,10,18,6,8,3,6,5,8,2,5,3,0,4,10,8,9,10,11,13,8,7,10,7,8,6,3,4,1,3,7,15,14,16,19,21,31,14,15,13,13,13,14,5,9,4,3,11,105,125,3,22,28,20,87,3,SOFISTICADOS
3,G-4,-16.442693,-71.544897,773446,C,16,0,0,16,0,0,6,0,0,6,0,0,0,0,24816,0,0,24816,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9282,30,672,3672,864,2064,312,1626,402,1302,510,876,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21612,0,0,9282,30,672,3672,864,2064,312,1626,402,1302,510,876,21612,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,2,2,2,0,0,0,1,0,0,0,0,0,1,2,0,1,2,2,2,3,0,0,0,2,0,0,0,0,0,2,6,10,0,0,0,0,0,0,NaN
4,G-5,-16.442693,-71.544897,773446,C,16,0,0,16,0,0,6,0,0,6,0,0,0,0,24816,0,0,24816,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9282,30,672,3672,864,2064,312,1626,402,1302,510,876,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21612,0,0,9282,30,672,3672,864,2064,312,1626,402,1302,510,876,21612,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,2,2,2,0,0,0,1,0,0,0,0,0,1,2,0,1,2,2,2,3,0,0,0,2,0,0,0,0,0,2,6,10,0,0,0,0,0,0,NaN


In [5]:

def preparar_variables_clustering(df):
    df_new = df.copy()

    # 1. Variables de estructura socioeconómica como proporciones
    df_new['prop_NSE_A'] = df_new['POB_A'] / df_new['POB']
    df_new['prop_NSE_E'] = df_new['POB_E'] / df_new['POB']
    df_new['tamano_hogar'] = df_new['POB'] / df_new['HOG']

    # 2. Variables de patrones de gasto como proporciones
    categorias_gasto = ['ALI', 'ALQ', 'SAL', 'EDU', 'REC', 'TRA', 'COM', 'RES', 'VES', 'BEB', 'MUE', 'BIE']
    for cat in categorias_gasto:
        df_new[f'prop_gasto_{cat}'] = df_new[f'GAS_{cat}'] / df_new['GAS']

    # 3. Variables demográficas
    df_new['prop_bebes'] = (df_new['GEN_0_4']) / df_new['POB']
    df_new['prop_ninos'] = (df_new['GEN_5_9'] + df_new['GEN_10_14']) / df_new['POB']
    df_new['prop_jovenes'] = (df_new['GEN_15_19'] + df_new['GEN_20_24'] + df_new['GEN_25_29']) / df_new['POB']
    df_new['prop_adultos'] = (
        df_new['GEN_30_34'] + df_new['GEN_35_39'] + df_new['GEN_40_44'] +
        df_new['GEN_45_49'] + df_new['GEN_50_54'] + df_new['GEN_55_59'] +
        df_new['GEN_60_64']
    ) / df_new['POB']
    df_new['prop_mayores'] = (
        df_new['GEN_65_69'] + df_new['GEN_70_74'] + df_new['GEN_75_79'] + df_new['GEN_80_MAS']
    ) / df_new['POB']

    # Ratio hombres/mujeres
    df_new['ratio_H_M'] = df_new['H'] / df_new['M']

    # 4. Variables de estilo de vida como proporciones
    estilos_vida = ['AUSTERO', 'FORMALISTA', 'MODERNAS', 'PROGRESIST', 'SOFISTICAD', 'CONSERVADO']


    for estilo in estilos_vida:
        df_new[f'prop_{estilo.lower()}'] = df_new[estilo] / df_new['POB']

    # Creamos las listas de columnas por categoría para facilitar su uso posterior
    dict_columnas = {
        'cols_estructura_socioeconomica': [
            'prop_NSE_A', 'prop_NSE_E', 'ING_PR', 'GAS_PR', 'tamano_hogar'
        ],

        'cols_patrones_gasto': [f'prop_gasto_{cat}' for cat in categorias_gasto],

        'cols_demograficas': [
            'prop_bebes','prop_ninos', 'prop_jovenes', 'prop_adultos', 'prop_mayores', 'ratio_H_M'
        ],

        'cols_estilos_vida': [f'prop_{estilo.lower()}' for estilo in estilos_vida],

        # Todas las columnas para clustering juntas
        'cols_clustering': []  # Se llenará a continuación
    }

    # Unimos todas las columnas para clustering
    dict_columnas['cols_clustering'] = (
        dict_columnas['cols_estructura_socioeconomica'] +
        dict_columnas['cols_patrones_gasto'] +
        dict_columnas['cols_demograficas'] +
        dict_columnas['cols_estilos_vida'] + df_new['gloria_id']
    )

    # Columnas a excluir del clustering (para referencia)
    dict_columnas['cols_excluir'] = [
        'ID_GLOR', 'PRED', 'PRED_EV',
        'POB_A', 'POB_B', 'POB_C', 'POB_D', 'POB_E',
        'HOG_A', 'HOG_B', 'HOG_C', 'HOG_D', 'HOG_E'
    ]

    return df_new , dict_columnas




In [6]:
df_new, dict_columnas = preparar_variables_clustering(df_src)
df_new.head()

ValueError: operands could not be broadcast together with shapes (29,) (152186,) 